### Setup

- pip install dash pandas lxml nb_black dash-bootstrap-components
- download card_list.csv from 17Lands
- save each set as a different page under one dir. Filename should be `data/{set}_card_ratings.html` https://17lands-public.s3.amazonaws.com/analysis_data/cards/card_list.csv
- start this jupyter notebook (making sure the card_list and set data are in `data/`)
- update the path to your log file (can be found in 17Lands client)
- go to `localhost:8050/` to see the app!

In [ ]:
# ! pip install nb_black
# ! pip install dash-bootstrap-components
%load_ext nb_black

In [ ]:
import requests
import os
import json
import logging
from typing import List, Sequence, Union, Optional, Dict, Any, Tuple
import datetime

import plotly
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash import dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from mtga_follower import (
    Follower,
    API_ENDPOINT,
    get_config,
    JSON_START_REGEX,
    extract_time,
    json_value_matches,
    get_rank_string,
    logger,
)

In [ ]:
class DashFollower(Follower):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pack_number = ""
        self.pick_number = ""

    def _Follower__retry_post(self, *args, **kwargs):
        """
        We don't want to mess anything up, so don't talk to the API at all
        """
        r = requests.Response()
        r.status_code = 200
        return r

    def _Follower__handle_bot_draft_pack(self, json_obj: Dict[str, Any]):
        super()._Follower__handle_bot_draft_pack(json_obj)
        self.pick_options = [int(x) for x in json_obj["DraftPack"]]

    def _Follower__handle_bot_draft_pick(self, json_obj: Dict[str, Any]):
        super()._Follower__handle_bot_draft_pick(json_obj)
        self.pack_number = json_obj["PackNumber"]
        self.pick_number = json_obj["PickNumber"]
        if not hasattr(self, "pool_card_ids"):
            self.pool_card_ids = []
        self.pool_card_ids.append(int(json_obj["CardId"]))

    def _Follower__handle_joined_pod(self, json_obj: Dict[Any, str]):
        self.pick_options = []
        self.pool_card_ids = []

    def _Follower__handle_blob(self, full_log):
        """Attempt to parse a complete log message and send the data if relevant."""
        match = JSON_START_REGEX.search(full_log)
        if not match:
            return
        try:
            json_obj, end = self.json_decoder.raw_decode(full_log, match.start())
        except json.JSONDecodeError as e:
            logger.debug(
                f"Ran into error {e} when parsing at {self.cur_log_time}. Data was: {full_log}"
            )
            return

        json_obj = self._Follower__extract_payload(json_obj)
        #         print(json_obj)
        if type(json_obj) != dict:
            return

        try:
            maybe_time = self._Follower__maybe_get_utc_timestamp(json_obj)
            if maybe_time is not None:
                self.last_utc_time = maybe_time
        except:
            pass

        if json_value_matches(
            "Client.Connected", ["params", "messageName"], json_obj
        ):  # Doesn't exist any more
            self._Follower__handle_login(json_obj)
        elif "Event_Join" in full_log and "EventName" in json_obj:
            #             print("joined event")
            self._Follower__handle_joined_pod(json_obj)
        elif "DraftStatus" in json_obj:
            #             print("draft status")
            self._Follower__handle_bot_draft_pack(json_obj)
        elif "BotDraft_DraftPick" in full_log and "PickInfo" in json_obj:
            #             print("draft pick")
            self._Follower__handle_bot_draft_pick(json_obj["PickInfo"])
        elif "LogBusinessEvents" in full_log and "PickGrpId" in json_obj:
            self._Follower__handle_human_draft_combined(json_obj)
        elif "Draft.Notify " in full_log and "method" not in json_obj:
            self._Follower__handle_human_draft_pack(json_obj)
        elif "authenticateResponse" in json_obj:
            self._Follower__update_screen_name(
                json_obj["authenticateResponse"]["screenName"]
            )

def read_data(
    data_path: str,
    supported_expansions: List[str] = ["ZNR", "KHM", "STX", "AFR", "MID", "VOW"],
) -> pd.DataFrame:
    """
    Load up the html version of the 17Lands card ratings and stack them. Also
    load the card_id mapping and join it
    """
    card_ids_df = pd.read_csv(
        os.path.join(data_path, "card_list.csv"),
        usecols=[
            "id",
            "expansion",
            "name",
            "rarity",
            "color_identity",
            "mana_value",
            "types",
        ],
    )
    card_ids_df = card_ids_df[card_ids_df.expansion.isin(supported_expansions)]
    card_ids_df = card_ids_df[["id", "expansion", "name", "types"]]

    ratings_df = None
    for set_name in supported_expansions:
        single_df = pd.read_html(
            os.path.join(data_path, f"{set_name.lower()}_card_ratings.html")
        )[0]
        single_df["expansion"] = set_name.upper()
        if ratings_df is None:
            ratings_df = single_df
        else:
            ratings_df = ratings_df.append(single_df)
    ratings_df = ratings_df.rename(columns={k: k.lower() for k in ratings_df.columns})

    joined = pd.merge(
        ratings_df,
        card_ids_df,
        how="left",
        left_on=["name", "expansion"],
        right_on=["name", "expansion"],
        suffixes=["", "_y"],
    )
    joined["iwd"] = joined["iwd"].apply(lambda x: float(str(x).replace("pp", "")))
    return joined.set_index("id")


In [ ]:
# logfile = r"C:\Users\thisi\AppData\LocalLow\Wizards Of The Coast\MTGA\Player.log"
# token = get_config()
# follower = DashFollower(token, API_ENDPOINT)

# r = follower.parse_log(logfile, follow=False)

In [ ]:
df = read_data("./data/")

In [ ]:
# number of seconds between refreshes. Needs to be long enough to fully parse your 
# .log file. If your file is huge, this could be an issue, and you may want to exit
# and reload the game
N_SECONDS = 10

# This is the path to your log file. If you want to test it out, I recommend
# saving a copy of one and editing it to have however many events you'd like
logfile = r"C:\Users\...\AppData\LocalLow\Wizards Of The Coast\MTGA\Player.log"

# This may pop up a dialog the first time, I'm not sure.
token = get_config()

# This controls what displays. you can print(df) to see your column options
useful_cols = [
    "name",
    "color",
    "types",
    "rarity",
    "alsa",
    "ata",
    "oh wr",
    "gd wr",
    "iwd",
]

# allows you to override the CSS of the dash app
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css", dbc.themes.GRID]

# modify this to change the cell styling in the tables
cell_styling = {"textAlign": "left", "padding": "5px"}

# the core of the app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    html.Div(
        [
            dbc.Container(
                [
                    dbc.Row(html.H4("Current Options")),
                    dbc.Row(html.Div(id="live-update-text")),
                    dbc.Row(html.Div(id="current-pick-table-holder")),
                    dbc.Row(html.H4("Pool So Far")),
                    dbc.Row(html.Div(id="pool-table-holder")),
                ]
            ),
            dcc.Interval(
                id="interval-component",
                interval=N_SECONDS * 1000,  # in milliseconds
                n_intervals=0,
            ),
        ]
    )
)


def get_color_count_graph(inp_df: pd.DataFrame) -> plotly.graph_objects.Figure:
    """
    Count the occurences of each "color" symbol in the card data's "color" column
    in the pool of cards you've drafted so far.
    """
    color_counts = (
        inp_df.color.apply(lambda x: list(x) if isinstance(x, str) else [])
        .explode()
        .value_counts()
        .reset_index()
        .rename(columns={"index": "Color", "color": "Count"})
    )
    color_hexes = ["#838383", "#26b569", "#f85656", "#aae0fa", "#fef2be"]
    missing_colors = [
        c for c in ("B", "G", "R", "U", "W") if c not in color_counts.Color.values
    ]
    missing_df = pd.DataFrame(
        [[c, 0] for c in missing_colors], columns=["Color", "Count"]
    )
    color_counts.append(missing_df)
    color_counts = color_counts.sort_values("Color")

    fig = go.Figure(
        data=[
            go.Bar(name=x[0], x=[x[0]], y=[x[1]], marker_color=color_hexes[i])
            for i, x in enumerate(color_counts.values)
        ],
    )
    fig.update_layout(
        title={
            "text": "Card Color Counts",
            "y": 0.9,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        }
    )
    return fig


def get_type_summary(inp_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Get counts of the core card types and of the other keywords in your 
    pool of drafted cards so far.
    """
    copied = inp_df.copy()
    card_types = [
        "Land",
        "Artifact",
        "Creature",
        "Enchantment",
        "Planeswalker",
        "Instant",
        "Sorcery",
    ]
    copied["keywords"] = copied["types"].apply(
        lambda x: [t.strip() for t in x.split(" ") if t.strip() not in ("", "-")]
    )
    vc = copied["keywords"].explode().value_counts().reset_index()
    main = (
        vc[vc["index"].apply(lambda x: x in card_types)]
        .copy()
        .rename(columns={"index": "type", "keywords": "count"})
    )
    other = (
        vc[~vc["index"].apply(lambda x: x in card_types)]
        .copy()
        .rename(columns={"index": "keyword", "keywords": "count"})
    )
    return main, other


def get_pool_summary(inp_df: pd.DataFrame):
    """
    Return all of the summary objects:
      - color count graph
      - type count df
      - keyword count df
    """
    color_count_graph = get_color_count_graph(inp_df)
    main_type_sum, other_type_sum = get_type_summary(inp_df)
    return color_count_graph, main_type_sum, other_type_sum


# every N seconds, our Interval fires, and we update most of the app
# by parsing the log
@app.callback(
    [
        Output("live-update-text", "children"),
        Output("current-pick-table-holder", "children"),
        Output("pool-table-holder", "children"),
    ],
    Input("interval-component", "n_intervals"),
)
def update_metrics(n):
    """
    Update the whole app by parsing the log file
    """
    # create a 17Lands client with the API interaction removed
    follower = DashFollower(token, API_ENDPOINT)
    # parse your whole log
    r = follower.parse_log(logfile, follow=False)
    
    style = {"padding": "5px", "fontSize": "16px"}
    # get the cards available to be picked
    picked_df = df.loc[getattr(follower, "pick_options", []), useful_cols].sort_values(
        "iwd", ascending=False
    )
    # get the cards you've already drafted
    pool_df = df.loc[getattr(follower, "pool_card_ids", []), useful_cols].sort_values(
        "iwd", ascending=False
    )
    # turn them both into Table objects
    pick_table = html.Div(
        dash_table.DataTable(
            id="pick_table",
            columns=[{"name": i, "id": i} for i in picked_df.columns],
            data=picked_df.to_dict("records"),
            style_cell=cell_styling,
        ),
        style={"width": "80%", "margin": "auto"},
    )
    pool_table = html.Div(
        dash_table.DataTable(
            id="pool_table",
            columns=[{"name": i, "id": i} for i in pool_df.columns],
            data=pool_df.to_dict("records"),
            style_cell=cell_styling,
        )
    )
    # get the summary statistics for your pool
    pool_summary, main_type_sum, other_type_sum = get_pool_summary(pool_df)
    main_type_sum = dash_table.DataTable(
        id="main_type_table",
        columns=[{"name": i, "id": i} for i in main_type_sum.columns],
        data=main_type_sum.to_dict("records"),
        style_cell=cell_styling,
    )
    other_type_sum = dash_table.DataTable(
        id="other_type_table",
        columns=[{"name": i, "id": i} for i in other_type_sum.columns],
        data=other_type_sum.to_dict("records"),
        style_cell=cell_styling,
    )
    # update the container with the summary info
    curr_div = dbc.Container(
        dbc.Row(
            [
                dbc.Col(pool_table, width={"size": 8}),
                dbc.Col(
                    [
                        dbc.Row(dcc.Graph(figure=pool_summary)),
                        dbc.Row(main_type_sum),
                        dbc.Row(html.Div("--")),
                        dbc.Row(other_type_sum),
                    ],
                    width={"size": 4},
                ),
            ]
        )
    )

    return [
        html.Span(
            f"Pack {follower.pack_number}, Pick {follower.pick_number}", style=style
        ),
        pick_table,
        curr_div,
    ]


if __name__ == "__main__":
    app.run_server()